## What this notebook will do 
This notebook will combine the split-measuring of observed and virtual audio files. Till now, the split-measuring have been done in separate notebooks - making it all a bit scattered. This is my attempt at getting it all into one place. 

### Jump to sections:

1. Split-measuring 
    * [Observed audio split measuring](#observed-audio) 
    * [Virtual audio split measuring](#virtual-audio)
1. [Keeping only 50 ms windows ](#only50ms)
1. [Choosing non-silent windows](#loudwindows)
1. [Assigning number of bats to each annotation](#assignbatnum)
1. [Saving non-silent window measurements](#savingdata)


Some of the things I'll make sure to include are:

* removal of the last audio window measurements. This is because the last audio window is very likely to be <50 ms. I'd set a threshold of between 45-50ms to maximise the number of windows included in the analysis, but after discussions with N, it just added to the variation in spectral and temporal resolution across windows. To simplify things, we decided it's just best to get rid of windows that are <50ms long. Either way, it's not like this exclusion of <50 ms windows will really affect sample sizes. 


Original date of notebook creation : 2020-11-21

Author: Thejasvi Beleyur

In [1]:
import datetime as dt
import glob
import sys 
sys.path.append('../../correct_call_annotations/')
sys.path.append('../')
sys.path.append('../../individual_call_analysis/analysis/')

import correct_call_annotations.correct_call_annotations as cca
import format_and_clean
from format_and_clean import ind_call_format as icf
import measure_annot_audio

from measure_annot_audio import split_measure
from measure_annot_audio.inbuilt_measurement_functions import dB
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import soundfile as sf
import tqdm

In [2]:
%matplotlib notebook

In [3]:
print(f'Notebook started at: {dt.datetime.now()}')

Notebook started at: 2020-12-17 15:16:40.586850


<a id='observed-audio'></a>

## Observed audio split-measuring

In [4]:
verif_annots = pd.read_csv('../verified_annotations.csv').reset_index(drop=True) # manually re-checked set of audio files.
# See ../choosing valid annotation audio files.ipynb

verif_annots

,Unnamed: 0,valid_annotations,video_annot_id,num_bats
0,0,matching_annotaudio_Aditya_2018-08-16_21502300_18,Aditya_2018-08-16_21502300_18,1
1,1,matching_annotaudio_Aditya_2018-08-16_21502300_20,Aditya_2018-08-16_21502300_20,1
2,2,matching_annotaudio_Aditya_2018-08-16_21502300_22,Aditya_2018-08-16_21502300_22,1
3,3,matching_annotaudio_Aditya_2018-08-16_21502300_24,Aditya_2018-08-16_21502300_24,1
4,4,matching_annotaudio_Aditya_2018-08-16_21502300_25,Aditya_2018-08-16_21502300_25,1
...,...,...,...,...
409,430,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_84,1
410,431,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_85,1
411,432,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_90,1
412,433,matching_annotaudio_Aditya_2018-08-20_0300-040...,Aditya_2018-08-20_0300-0400_91,1


In [5]:
all_measures = []

In [6]:
fs = 250000
kwargs = { 'inter_peak_difference':250, 
           'spectrum_smoothing_width': 100,
           'peak_range': 14,
           'fs':fs,
           'db_range':46,
           'overlap':0.5,
           'threshold':95,
           'nperseg':512,
           'fm_freqs_condition' : lambda X : X<98000,
           'non_signal_freqs_condition' : lambda X: X<70000
          
            }

In [7]:
for i, row in tqdm.tqdm(verif_annots.iterrows()):
    
    audio_folder = '../../individual_call_analysis/hp_annotation_audio/'
    file_path = cca.find_file_in_folder(verif_annots['valid_annotations'][i]+'_hp.WAV', audio_folder)
    measurements = split_measure.split_measure_audio(file_path[0],
                                  **kwargs)
    all_measures.append(measurements)


0it [00:00, ?it/s]

Match found!


1it [00:00,  2.64it/s]

Match found!


2it [00:01,  1.74it/s]

...

Match found!
Match found!


413it [03:21,  4.00it/s]

414it [03:21,  4.34it/s]

414it [03:21,  2.05it/s]

Match found!


In [8]:
all_split_measure = pd.concat(all_measures).reset_index(drop=True)

In [9]:
all_split_measure

,value,segment_number,measurement,file_name
0,0.045400,0,rms,matching_annotaudio_Aditya_2018-08-16_21502300...
1,0.118805,0,peak_amplitude,matching_annotaudio_Aditya_2018-08-16_21502300...
2,105660.000000,0,dominant_frequencies,matching_annotaudio_Aditya_2018-08-16_21502300...
3,87402.343750,0,fm_terminal_freqs,matching_annotaudio_Aditya_2018-08-16_21502300...
4,85449.218750,0,fm_terminal_freqs,matching_annotaudio_Aditya_2018-08-16_21502300...
...,...,...,...,...
402688,103600.000000,12,dominant_frequencies,matching_annotaudio_Aditya_2018-08-20_0300-040...
402689,103840.000000,12,dominant_frequencies,matching_annotaudio_Aditya_2018-08-20_0300-040...
402690,104540.000000,12,dominant_frequencies,matching_annotaudio_Aditya_2018-08-20_0300-040...
402691,105100.000000,12,dominant_frequencies,matching_annotaudio_Aditya_2018-08-20_0300-040...


In [10]:
all_split_measure['unique_window_id'] = all_split_measure['segment_number'].astype(str) +'_'+all_split_measure['file_name'] 

<a id='virtual-audio'></a>
## Virtual audio split-measuring 

In [11]:
audio_source_folder = '../virtual_multi_bat_audio/'
virtual_multibat_to_measure = glob.glob(audio_source_folder + '*.WAV')

In [12]:
all_virt_measures = []
for each in tqdm.tqdm(virtual_multibat_to_measure):

    measurements = split_measure.split_measure_audio(each,
                                  **kwargs)
    all_virt_measures.append(measurements)


  0%|                                                                                           | 0/89 [00:00<?, ?it/s]

  1%|▉                                                                                  | 1/89 [00:00<00:14,  5.91it/s]

  2%|█▊                                                                                 | 2/89 [00:01<00:34,  2.52it/s]

  3%|██▊                                                                                | 3/89 [00:02<00:48,  1.76it/s]

  4%|███▋                                                                               | 4/89 [00:02<00:41,  2.03it/s]

...

 96%|██████████████████████████████████████████████████████████████████████████████▎   | 85/89 [00:28<00:02,  1.90it/s]

 97%|███████████████████████████████████████████████████████████████████████████████▏  | 86/89 [00:28<00:01,  1.97it/s]

 98%|████████████████████████████████████████████████████████████████████████████████▏ | 87/89 [00:29<00:00,  2.25it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [00:30<00:00,  1.82it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [00:30<00:00,  2.90it/s]

In [13]:
all_virtual_splitmeasure = pd.concat(all_virt_measures).reset_index(drop=True)

all_virtual_splitmeasure['unique_window_id'] = all_virtual_splitmeasure['segment_number'].astype(str) +'_'+all_virtual_splitmeasure['file_name'] 


In [14]:
virt_rms_values = all_virtual_splitmeasure[all_virtual_splitmeasure['measurement']=='rms']
obs_rms_values = all_split_measure[all_split_measure['measurement']=='rms']
plt.figure()
plt.violinplot([dB(obs_rms_values['value']) , dB(virt_rms_values['value'])], quantiles=[[0.025,0.5,0.975]]*2)
plt.title('Window intensities')
plt.xticks([1,2],['Observed\nwindows','Virtual\nwindows'], fontsize=10)
plt.ylabel('Received level, dB rms', fontsize=12)

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Received level, dB rms')

Both observed and virtual audio windows have a wide range of levels. We now need to filter out those which are very silent, and unlikely to have calls or only have very faint calls. The threshold has been chosen as being +20 dB above the level of manually chosen silent audio segments. 

<a id='only50ms'></a>
## Keeping only 50 ms windows 
This is done by removing the last measured window from each audio file. The last window of each audio file can be 47-50ms due to the 90% allowance built into the ```split_measure.split_audio``` function. 

In [15]:
byfilename = all_split_measure.groupby('file_name')

only50ms_windows_allfiles = []
for filename, df in byfilename:
    segment_numbers = np.sort(np.unique(df['segment_number']))
    
    if len(segment_numbers)>1:
        only50ms_windows = df[df['segment_number']!=segment_numbers[-1]]
        only50ms_windows_allfiles.append(only50ms_windows)
    else:
        only50ms_windows_allfiles.append(df)
obs_50ms_splitmeasure = pd.concat(only50ms_windows_allfiles).reset_index(drop=True)

In [16]:
virt_byfilename = all_virtual_splitmeasure.groupby('file_name')

virt_only50ms_windows_allfiles = []
for filename, df in virt_byfilename:
    segment_numbers = np.sort(np.unique(df['segment_number']))
    
    if len(segment_numbers)>1:
        only50ms_windows = df[df['segment_number']!=segment_numbers[-1]]
        virt_only50ms_windows_allfiles.append(only50ms_windows)
    else:
        virt_only50ms_windows_allfiles.append(df)

virt_50ms_splitmeasure = pd.concat(virt_only50ms_windows_allfiles).reset_index(drop=True)

<a id='loudwindows'></a>
##  Choosing non-silent windows

Let's load the silent audio +20 dB thresholds to choose only those windows that are loud. 

In [17]:
plus20dB_threshold = pd.read_csv('../call_threshold_levels.csv')
plus20dB_threshold

,Unnamed: 0,call_dbpeak_threshold,call_dbrms_threshold
0,0,-23.0,-37.0


In [18]:
def choose_all_windows_above_threshold(measurement_df, threshold):
    '''
    Chooses all windows with rms greater or equal to the given rms threshold
    
    
    Parameters
    ----------
    measurement_df : pd.DataFrame
        A long dataframe with one measurement value per row
        Must have the columns 'measurement', 'unique_window_id'
        The 'measurement' column must also have some entries with 'rms' in them. 
    threshold : float>0
        The rms threshold to define windows which are not silent. 
    
    Returns 
    -------
    all_above_threshold : pd.DataFrame
        The dataframe with all segments across different files that are greater than or equal to the 
        chosen threshold rms. 
    '''

    rms_values = measurement_df[measurement_df['measurement']=='rms'].reset_index(drop=True)
    above_threshold = rms_values[rms_values['value']>=threshold]
    windowids_above_threshold = above_threshold['unique_window_id']
    all_above_threshold = measurement_df[measurement_df['unique_window_id'].isin(windowids_above_threshold)].reset_index(drop=True)
    return all_above_threshold


In [19]:
threshold_20db = float(10**(plus20dB_threshold['call_dbrms_threshold']/20.0))
threshold_20db

0.01412537544622754

In [20]:
non_silent_observedaudio_measures = choose_all_windows_above_threshold(obs_50ms_splitmeasure, threshold_20db)
non_silent_virtualaudio_measures = choose_all_windows_above_threshold(virt_50ms_splitmeasure, threshold_20db)


In [21]:
rms_df = non_silent_observedaudio_measures[non_silent_observedaudio_measures['measurement']=='rms']
virt_rmsdf = non_silent_virtualaudio_measures[non_silent_virtualaudio_measures['measurement']=='rms']

plt.figure()
plt.violinplot([dB(rms_df['value']), dB(virt_rmsdf['value'])], showextrema=False, 
               quantiles=[[0.025,0.5,0.975]]*2);
plt.xticks([1,2],['Observed windows','Virtual windows']);plt.ylabel('Non-silent window received level, dB rms', fontsize=12)

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Non-silent window received level, dB rms')

<a id='assignbatnum'></a>
## Assigning number of bats to each annotation

In [22]:
# observed audio bat number assigning

non_silent_observedaudio_measures['video_annot_id'] = non_silent_observedaudio_measures['file_name'].str.lstrip('matching_annotaudio_')
# remove the '_hp' from the video_annot_ida
non_silent_observedaudio_measures['video_annot_id'] = non_silent_observedaudio_measures['video_annot_id'].str.rstrip('_hp.WAV')

# match the annotation id to the number of bats 
video_annot_folder = '../../whole_data_analysis/annotations/corrected_HBC_video_annotations_Aditya/'
non_silent_observedaudio_measures['num_bats'] = icf.get_numbats_from_annotation_id(non_silent_observedaudio_measures['video_annot_id'], video_annot_folder)

In [23]:
non_silent_observedaudio_measures

,value,segment_number,measurement,file_name,unique_window_id,video_annot_id,num_bats
0,0.022077,0,rms,matching_annotaudio_Aditya_2018-08-16_21502300...,0_matching_annotaudio_Aditya_2018-08-16_215023...,Aditya_2018-08-16_21502300_100,1
1,0.062134,0,peak_amplitude,matching_annotaudio_Aditya_2018-08-16_21502300...,0_matching_annotaudio_Aditya_2018-08-16_215023...,Aditya_2018-08-16_21502300_100,1
2,105040.000000,0,dominant_frequencies,matching_annotaudio_Aditya_2018-08-16_21502300...,0_matching_annotaudio_Aditya_2018-08-16_215023...,Aditya_2018-08-16_21502300_100,1
3,90332.031250,0,fm_terminal_freqs,matching_annotaudio_Aditya_2018-08-16_21502300...,0_matching_annotaudio_Aditya_2018-08-16_215023...,Aditya_2018-08-16_21502300_100,1
4,89843.750000,0,fm_terminal_freqs,matching_annotaudio_Aditya_2018-08-16_21502300...,0_matching_annotaudio_Aditya_2018-08-16_215023...,Aditya_2018-08-16_21502300_100,1
...,...,...,...,...,...,...,...
17933,0.044647,3,peak_amplitude,matching_annotaudio_Aditya_2018-08-20_0300-040...,3_matching_annotaudio_Aditya_2018-08-20_0300-0...,Aditya_2018-08-20_0300-0400_91,1
17934,105200.000000,3,dominant_frequencies,matching_annotaudio_Aditya_2018-08-20_0300-040...,3_matching_annotaudio_Aditya_2018-08-20_0300-0...,Aditya_2018-08-20_0300-0400_91,1
17935,87402.343750,3,fm_terminal_freqs,matching_annotaudio_Aditya_2018-08-20_0300-040...,3_matching_annotaudio_Aditya_2018-08-20_0300-0...,Aditya_2018-08-20_0300-0400_91,1
17936,85449.218750,3,fm_terminal_freqs,matching_annotaudio_Aditya_2018-08-20_0300-040...,3_matching_annotaudio_Aditya_2018-08-20_0300-0...,Aditya_2018-08-20_0300-0400_91,1


In [24]:
# report the stats on number of annotations with diff types of bats
print('Number of observed annotations by group size')
obs_by_batnum = non_silent_observedaudio_measures.groupby(['num_bats'])
for batnum, df in obs_by_batnum:
    print(batnum, np.unique(df['file_name']).size)

Number of observed annotations by group size
1 236
2 71
3 18
4 6


In [25]:
# virtual audio bat number assigning
non_silent_virtualaudio_measures['video_annot_id'] = non_silent_virtualaudio_measures['file_name'].str.lstrip('matching_annotaudio_')
# remove the '_hp' from the video_annot_ida
non_silent_virtualaudio_measures['video_annot_id'] = non_silent_virtualaudio_measures['video_annot_id'].str.rstrip('_hp_singlebatmixed.WAV')
video_annot_folder = '../../whole_data_analysis/annotations/corrected_HBC_video_annotations_Aditya/'
non_silent_virtualaudio_measures['num_bats'] = icf.get_numbats_from_annotation_id(non_silent_virtualaudio_measures['video_annot_id'], video_annot_folder)


In [26]:
# report the stats on number of annotations with diff types of bats
print('Number of virtual annotations by group size')
obs_by_batnum = non_silent_virtualaudio_measures.groupby(['num_bats'])
for batnum, df in obs_by_batnum:
    print(batnum, np.unique(df['file_name']).size)

Number of virtual annotations by group size
2 65
3 14
4 4


<a id='savingdata'></a>
## Saving non-silent window measurements

In [27]:
# observed audio data 
non_silent_observedaudio_measures.to_csv('obs_nonsilent_measurements_20dBthreshold.csv')

# virtual audio data 
non_silent_virtualaudio_measures.to_csv('virt_nonsilent_measurement_20dBthreshold.csv')

In [28]:
print(f'Notebook run ended at : {dt.datetime.now()}')

Notebook run ended at : 2020-12-17 15:20:58.546331
